# Introduction

1. Load in the camera parameters to calculate the distortion parameters.
2. Load in the board positions.
3. Calculate the projection matrices

Begin by building a dictionary that consolidates all of the calibration data collected.

In [98]:
import json
from pathlib import Path

cam0_cal = "calibration_params\cam_0.json"
cam1_cal = "calibration_params\cam_1.json"
stereocam_0_1_cal = "calibration_params\stereo_cam_0_cam_1.json"

calibration_data = {
                    "cam0": cam0_cal, 
                    "cam1": cam1_cal, 
                    "stereocam": stereocam_0_1_cal
                    }

for key, json_path in calibration_data.items():
    # print(key, json_path)

    with open(Path(Path.cwd(), json_path)) as f:
        calibration_data[key] = json.load(f)
        # print(calibration_data[key].keys())

# Define the Direct Linear Transformation
Isolate the parameters needed to create the projection matrices. Use them to define the Direct Linear Transformation that will be used to map the pair of 2D images to a 3D estimate. 

In [160]:
# here u and v are used by convention to indicate the x,y position of the detected point in a 2D plane.

import numpy as np

# cam0 orientation will define the global frame of reference, so
# RT1 is this...not fully sure what this stuff means
RT1 = np.concatenate([np.eye(3), [[0],[0],[0]]], axis = -1)
mtx1 = calibration_data["cam0"]["camera_matrix"]
P1 = mtx1 @ RT1 #projection matrix for C1


R2 = calibration_data["stereocam"]["rotation_vector"]
T2 = calibration_data["stereocam"]["translation_vector"]
RT2 = np.concatenate([R2, T2], axis = -1)
mtx2 = calibration_data["cam1"]["camera_matrix"]
P2 = mtx2 @ RT2 #projection matrix for C2


def triangulate(P1, P2, u_A, v_A, u_B, v_B):
 
    point1 = np.array([u_A, v_A])
    point2 = np.array([u_B, v_B])
    A = [point1[1]*P1[2,:] - P1[1,:],
         P1[0,:] - point1[0]*P1[2,:],
         point2[1]*P2[2,:] - P2[1,:],
         P2[0,:] - point2[0]*P2[2,:]
        ]
    A = np.array(A).reshape((4,4))
 
    B = A.transpose() @ A
    from scipy import linalg
    U, s, Vh = linalg.svd(B, full_matrices = False)
 
    return Vh[3,0:3]/Vh[3,3]


# Reconfigure the data

For ease of processing and plotting, convert the observed 2D image data into a consolidated dataframe.

In [158]:
# trying a different way to get a list of lists in the condition that I want

import pandas as pd

imgpointsA = calibration_data["stereocam"]["image_points_A"]
imgpointsB = calibration_data["stereocam"]["image_points_B"]
imgpointsID = calibration_data["stereocam"]["image_points_ID"]

frame = 0
summarized_points = []

for ID, pointA, pointB in zip(imgpointsID, imgpointsA, imgpointsB):
    
    for i, uv_A, uv_B in zip(ID, pointA, pointB):

        summarized_points.append([frame, int(i), uv_A[0], uv_A[1], uv_B[0], uv_B[1]])

    # print(frame)
    frame = frame + 1

df_pnts = pd.DataFrame(summarized_points,
                               columns = ["frame", "ID", "u_A", "v_A", "u_B", "v_B"])


# Estimate 3D Position

Traverse the dataframe and apply the DLT to each pair of points using the projection matrices calculated above

In [161]:

# see: https://stackoverflow.com/a/63832539/13563258
# this is a madlad answer right here

df_pnts[["X", "Y", "Z"]] = [triangulate(P1, P2, *a) for a in tuple(zip(df_pnts["u_A"], df_pnts["v_A"],df_pnts["u_B"],df_pnts["v_B"]))]




# Inspect results for reasonability



In [175]:
import plotly.express as px


fig = px.scatter_3d(df_pnts, x='X', y='Y', z='Z',
              color='frame')
fig.show()

Things appear to be sitting in a plane nicely, but when viewed more closely there is a warping of the position of the corners which may be due to a lack of correction for distortion in the camera.

In [176]:
fig = px.scatter_3d(df_pnts[df_pnts["frame"]==5], x='X', y='Y', z='Z',
              color='ID')
fig.show()

See tutorial [here](https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html) for details regarding how to undistort an image. I suspect that this may be something where it is preferable to go back to the source (i.e. perform a new calibration).